# Build synthetic data

In [2]:
import json
import pandas as pd

with open("./webtext2019zh/web_text_zh_train_sample.json", "r") as f:
    content = f.readlines()

content = map(json.loads, content)
content = pd.DataFrame(content)

In [6]:
text = "\n".join(content.content.values)

In [10]:
%%time
import re

def cut_sentences(text, min_len=3):
    """
    Cut sentences by their length and punctuation
    """
    corpus = re.split("[\,\.\?，。？\n]", text)
    corpus = list(filter(lambda x: len(x) >= min_len, corpus))
    return corpus

sentences = cut_sentences(text)

CPU times: user 558 µs, sys: 0 ns, total: 558 µs
Wall time: 564 µs


In [20]:
sentences[:10]

['我对你仍有爱意',
 '我对自己无能为力',
 '讲一个同学的事儿吧 那年他小学二年级 数学课 老师讲课的时候 同学跟同桌一块儿玩儿 小孩儿贪玩也很寻常 不寻常的是数学老师勃然大怒 抓起我同学的衣领 拎着他就直接走出教室 教室是在四楼 这老师竟然直接就把他悬在了走道护杆外的半空 同学直接吓尿了（是真的尿）这老师还在骂骂咧咧的威胁以后还敢不敢上课瞎捣蛋 结局同学还是安然被“收了回来” 但从此他不敢在数学课上放肆 从此他的数学成绩没有突破过及格线 一个好的老师 传',
 '我看了上帝之眼感觉还不错',
 '不喜欢那些讲技术的摄影书',
 ' 喜欢美用诗意和哲理来表现',
 '汉子已经被我收啦 现在是男票了哈哈哈哈哈 放两张聊天记录你们感受下',
 '看到这一幕我简直兴奋得要上天了 更新于上一个答案几个小时后 剧透醒目 论这个世界上还有什么比妖怪夫妇联手更恐怖的事',
 '小时候喜欢芭比娃娃',
 '太贵买不起']

# Mask filling
Inserting masks at random positions in the Wikipedia sentences, and fill them with the language model.

In [54]:
random.choices(range(10), k=2)

[7, 0]

In [295]:
%%time
import random
import math

s = "看到这一幕我简直兴奋得要上天了 更新于上一个答案几个小时后 剧透醒目 论这个世界上还有什么比妖怪夫妇联手更恐怖的事"

def mask_filling(s):
    """
    The first strategy samples random wordsin the sentence and it replaces them with masks(one for each token).
    """
    seq = list(s)
    seq_len = len(s)
    # Sample from 1 to half chars of the sequence
    k = random.randint(1, math.floor(seq_len * 0.75))
    token_idx = random.choices(range(seq_len), k = k)
    for i in token_idx:
        seq[i] = "<MASK>"
    return "".join(seq)

def mask_filling2(s):
    """
    The second strategy cre-ates contiguous sequences: 
    it samples a start po-sition s, a length l (uniformly distributed), 
    and it masks all the tokens spanned by words betweenpositions s and s + l.
    """
    seq_len = len(s)
    start = random.randint(1, seq_len-1)
    # At least mask 15% words, greater than 1
    min_length = min(math.floor(seq_len * 0.15), seq_len - start)
    min_length = max(min_length, 1)
    # At most 85% of words
    max_length = min(math.floor(seq_len * 0.85), seq_len - start)
    max_length = max(min_length, max_length)
    length = random.choice(range(min_length, max_length+1))
    
    s = s[:start] + "<MASK>" * length + s[(start+length):]
    return s

pd.Series(sentences).apply(mask_filling2).head()

CPU times: user 5.19 ms, sys: 0 ns, total: 5.19 ms
Wall time: 5.15 ms


0                                    我对你仍有<MASK><MASK>
1                              我对自<MASK><MASK><MASK>为力
2    讲一个同学的事儿吧 那年他小学二年级 数学课 老师讲课的时候 同学跟同桌一块儿玩儿 小孩儿贪...
3                          我看了上帝之眼<MASK><MASK><MASK>不错
4                               不喜欢那些讲技<MASK><MASK>摄影书
dtype: object

In [116]:
# Use GPT2 to fill in the missing fields


# Backtranslation
Translate chinese to english, and translate back.

# Word dropping